In [1]:
#Importing the dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

In [2]:
#Reading the CAvideo file into a dataframe
csv_file = "CAvideos.csv"
videoCA_df = pd.read_csv(csv_file)
videoCA_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [3]:
#Dropping the unnecessary columns
new_videoCA_df=videoCA_df[["video_id","title","category_id","publish_time","views","likes","dislikes","comment_count"]].copy()
new_videoCA_df.head()

,video_id,title,category_id,publish_time,views,likes,dislikes,comment_count
0,n1WpP7iowLc,Eminem - Walk On Water (Audio) ft. Beyoncé,10,2017-11-10T17:00:03.000Z,17158579,787425,43420,125882
1,0dBIkQ4Mz1M,PLUSH - Bad Unboxing Fan Mail,23,2017-11-13T17:00:00.000Z,1014651,127794,1688,13030
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,2017-11-12T19:05:24.000Z,3191434,146035,5339,8181
3,d380meD0W0M,I Dare You: GOING BALD!?,24,2017-11-12T18:01:41.000Z,2095828,132239,1989,17518
4,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),10,2017-11-09T11:04:14.000Z,33523622,1634130,21082,85067


In [4]:
#Renaming the columns
new_videoCA_df.rename(columns={"publish_time":"publish_datetime","comment_count":"comments"},inplace= True)
new_videoCA_df.head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments
0,n1WpP7iowLc,Eminem - Walk On Water (Audio) ft. Beyoncé,10,2017-11-10T17:00:03.000Z,17158579,787425,43420,125882
1,0dBIkQ4Mz1M,PLUSH - Bad Unboxing Fan Mail,23,2017-11-13T17:00:00.000Z,1014651,127794,1688,13030
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,2017-11-12T19:05:24.000Z,3191434,146035,5339,8181
3,d380meD0W0M,I Dare You: GOING BALD!?,24,2017-11-12T18:01:41.000Z,2095828,132239,1989,17518
4,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),10,2017-11-09T11:04:14.000Z,33523622,1634130,21082,85067


In [5]:
#Importing langdetect library to drop the titles not in english language
from langdetect import detect, DetectorFactory
DetectorFactory.seed=0

In [6]:
CA_df=new_videoCA_df.copy()

In [7]:
len(new_videoCA_df["title"].unique())


24573

In [8]:
len(CA_df)

40881

In [9]:
#Dropping the duplicates
CA_df.drop_duplicates(subset=["title"],keep = "last",inplace=True)

In [10]:
len(CA_df)

24573

In [11]:
#Dropping titles not in english language and also recording the error titles to be dropped later 
err_title=[]
row_index=[]
for i, row in CA_df.iterrows():
    try: 
        review = row['title']
        lang = detect(str(review))
        if lang!='en':
            CA_df.drop(i, inplace=True)
    
    except:
        lang = "error"
        err_title.append(row['title'])
        row_index.append(i)
        print("This row throws an error:", row['title'])

This row throws an error: LTV | ልዩ የበዓል ፕሮግራም ከተውዳጇ ድምጻዊት ብፅአት ስዩም ጋር ...
This row throws an error: Ethiopia ሰበር ዜና መንግስት የአስቸኳይ ጊዜ አዋጅ አወጀ
This row throws an error: የአዲስ አበባ ነዋሪ የዶ/ር አብይን መመረጥ በማስመልከት ከተማዋን በጭፈራ ሲያምስዋት
This row throws an error: በጎንደር ጠ/ሚ ዶ/ር አብይ አህመድ ከጎንደር ነዋሪ ተወካዮች የቀረቡላቸው ጥያቄዎች እና የሰጡት ምላሽ
This row throws an error: Dr abiy ahmed: እንደ ኦሮሞ የምታዩኝ ከሆነ ተሳስታቿል ጠ/ሚ አብይ
This row throws an error: ኢሳት ቲቪ በአዲስ አበባ ቢሮ እንዲከፍት ተፈቀደለት
This row throws an error: ጠ/ሚ ዓብይ አህመድ ኬንያ ሲገቡ ያጋጠማጭቸውን ይመልከቱ
This row throws an error: 이상형 10명 한번에 만나기
This row throws an error: ⠑⠥⠟⠊⠗⠑⠝⠑⠛⠤⠥⠁⠑⠧⠥⠕⠝⠤⠎⠑⠊⠗⠪⠓⠞⠤⠟⠝⠊⠉⠲⠍⠏⠠⠹
This row throws an error: Ethiopia: ጠሚንስቴር አብይ ዛሬ አሳዛኝ ስብሰባላይ አደረጉ
This row throws an error: 435


In [12]:
row_index

[11180, 18933, 26512, 30114, 30315, 31405, 33249, 33709, 33859, 36652, 37072]

In [13]:
err_title

['LTV | ልዩ የበዓል ፕሮግራም ከተውዳጇ ድምጻዊት ብፅአት ስዩም ጋር ...',
 'Ethiopia ሰበር ዜና መንግስት የአስቸኳይ ጊዜ አዋጅ አወጀ',
 'የአዲስ አበባ ነዋሪ የዶ/ር አብይን መመረጥ በማስመልከት ከተማዋን በጭፈራ ሲያምስዋት',
 'በጎንደር ጠ/ሚ ዶ/ር አብይ አህመድ ከጎንደር ነዋሪ ተወካዮች የቀረቡላቸው ጥያቄዎች እና የሰጡት ምላሽ',
 'Dr abiy ahmed: እንደ ኦሮሞ የምታዩኝ ከሆነ ተሳስታቿል ጠ/ሚ አብይ',
 'ኢሳት ቲቪ በአዲስ አበባ ቢሮ እንዲከፍት ተፈቀደለት',
 'ጠ/ሚ ዓብይ አህመድ ኬንያ ሲገቡ ያጋጠማጭቸውን ይመልከቱ',
 '이상형 10명 한번에 만나기',
 '⠑⠥⠟⠊⠗⠑⠝⠑⠛⠤⠥⠁⠑⠧⠥⠕⠝⠤⠎⠑⠊⠗⠪⠓⠞⠤⠟⠝⠊⠉⠲⠍⠏⠠⠹',
 'Ethiopia: ጠሚንስቴር አብይ ዛሬ አሳዛኝ ስብሰባላይ አደረጉ',
 '435']

In [14]:
len(CA_df)

17189

In [15]:
#Dropping the error titles from dataframe
for i in row_index:
    CA_df.drop(index=i,axis=0,inplace=True)

In [16]:
len(CA_df)

17178

In [17]:
#Storing back the modified dataframe in the new_videoCA_df
new_videoCA_df=CA_df

In [18]:
len(new_videoCA_df)

17178

In [19]:
#Reading the USvideos file in a dataframe
csv_file = "USvideos.csv"
videoUS_df = pd.read_csv(csv_file)
videoUS_df.head(1)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...


In [20]:
#DRopping the columns not required and renaming the columns
new_videoUS_df = videoUS_df[['video_id', 'title', 'category_id', 'publish_time', 'views', 'likes', 'dislikes', 'comment_count']].copy()
new_videoUS_df = new_videoUS_df.rename(columns = {"publish_time": "publish_datetime", "comment_count": "comments"})

new_videoUS_df.head(1)

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,22,2017-11-13T17:13:01.000Z,748374,57527,2966,15954


In [21]:
#Dropping the Duplicates
new_videoUS_df.drop_duplicates(subset = 'title', keep ='last', inplace=True)

In [22]:
### drop non-english language 


for i, row in new_videoUS_df.iterrows():
    try:
        lang = detect(str(row["title"]))
        if lang !="en":
            new_videoUS_df.drop(i, inplace = True)
    except:
        lang = "error"


In [23]:
len(new_videoUS_df) 

5575

In [24]:
#Reading the json file into the dataframe

json_file = "US_category_id.json"
category_df = pd.read_json(json_file)
category_df.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [25]:
len(category_df)

32

In [26]:
category_df["items"][0]

{'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
 'id': '1',
 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title': 'Film & Animation',
  'assignable': True}}

In [27]:
items_list=category_df["items"].tolist()

In [28]:
#Extracting the required information from the category dataframe
id_list=[]
title=[]
for item in items_list:
    id_list.append(int(item["id"]))
    title.append(item["snippet"]["title"])
    

In [30]:
#title

In [29]:
#Storing the information in a dataframe
new_category_df=pd.DataFrame({"category_title":title,"category_id":id_list})

In [30]:
new_category_df.head()

,category_title,category_id
0,Film & Animation,1
1,Autos & Vehicles,2
2,Music,10
3,Pets & Animals,15
4,Sports,17


In [31]:
len(new_category_df)

32

In [32]:
new_category_df.dtypes
new_category_df["category_id"]=new_category_df["category_id"].astype(int)
new_category_df.dtypes

category_title    object
category_id        int32
dtype: object

In [33]:
#Merging the individual country dataframes with the category dataframe
mergedCA_df=pd.merge(new_videoCA_df,new_category_df,on="category_id",how="inner")
mergedCA_df.tail()

mergedUS_df=pd.merge(new_videoUS_df,new_category_df,on="category_id",how="inner")
mergedUS_df.head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments,category_title
0,9wRQljFNDW8,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,17,2017-11-13T02:05:26.000Z,81377,655,25,177,Sports
1,fIQ2Ty2OL34,James Harden Triple-Double 2017.11.09 vs Cavs ...,17,2017-11-10T04:00:52.000Z,18573,420,10,236,Sports
2,agKDPntMv-E,Stephen A. Smith slams the Philadelphia 76ers ...,17,2017-11-08T15:59:06.000Z,470191,4037,430,1553,Sports
3,Xl38SuxyrgM,"STEPH CURRY and OMRI CASSPI, postgame GSW (9-3...",17,2017-11-09T07:27:32.000Z,16305,147,13,47,Sports
4,0zk2SwmCpss,Joel Embiid Full Highlights 2017.11.13 at Clip...,17,2017-11-14T06:24:27.000Z,63780,831,12,301,Sports


In [34]:
#Adding a country column to the merged dataframes
mergedCA_df["country"]="CA"
mergedUS_df["country"]="US"

In [35]:
mergedCA_df.head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments,category_title,country
0,0yIWz1XEeyc,Jake Paul Says Alissa Violet CHEATED with LOGA...,25,2017-11-13T07:37:51.000Z,1309699,103755,4613,12143,News & Politics,CA
1,TIvI07xLPnA,"The National for Sunday, November 12, 2017",25,2017-11-13T03:06:10.000Z,13433,74,57,0,News & Politics,CA
2,RbcGCMtlluQ,Final Nail In The Coffin: SNL Turns Against Th...,25,2017-11-13T05:46:39.000Z,116344,5068,106,1341,News & Politics,CA
3,TUmyygCMMGA,Why the rise of the robots won’t mean the end ...,25,2017-11-13T13:45:16.000Z,256426,12656,1363,2368,News & Politics,CA
4,2B8RO0GcLDw,Live with Dr.Shahid Masood | Asif Zardari | Na...,25,2017-11-12T16:23:14.000Z,187075,2443,185,324,News & Politics,CA


In [36]:
mergedUS_df.head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments,category_title,country
0,9wRQljFNDW8,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,17,2017-11-13T02:05:26.000Z,81377,655,25,177,Sports,US
1,fIQ2Ty2OL34,James Harden Triple-Double 2017.11.09 vs Cavs ...,17,2017-11-10T04:00:52.000Z,18573,420,10,236,Sports,US
2,agKDPntMv-E,Stephen A. Smith slams the Philadelphia 76ers ...,17,2017-11-08T15:59:06.000Z,470191,4037,430,1553,Sports,US
3,Xl38SuxyrgM,"STEPH CURRY and OMRI CASSPI, postgame GSW (9-3...",17,2017-11-09T07:27:32.000Z,16305,147,13,47,Sports,US
4,0zk2SwmCpss,Joel Embiid Full Highlights 2017.11.13 at Clip...,17,2017-11-14T06:24:27.000Z,63780,831,12,301,Sports,US


In [39]:
#Establishing a connection with postgres SQL
rds_connection_string = f"{username}:{password}@localhost:5432/youtube_video_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [40]:
engine.table_names()

['category', 'video_ca', 'video_us', 'video_category_ca', 'video_category_us']

In [41]:
#Loading the dataframes in the SQL database
new_category_df.to_sql(name='category', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from category', con=engine).head()

,category_title,category_id
0,Film & Animation,1
1,Autos & Vehicles,2
2,Music,10
3,Pets & Animals,15
4,Sports,17


In [42]:
new_videoCA_df.to_sql(name='video_ca', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from video_ca', con=engine).head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments
0,0yIWz1XEeyc,Jake Paul Says Alissa Violet CHEATED with LOGA...,25,2017-11-13T07:37:51.000Z,1309699,103755,4613,12143
1,FyZMnhUtLfE,猎场 | Game Of Hunting 12【TV版】（胡歌、張嘉譯、祖峰等主演）,1,2017-11-12T16:00:01.000Z,158815,218,30,186
2,gifPYwArCVQ,Fake Pet Smart Employee Prank!,23,2017-11-13T01:30:01.000Z,557883,44558,621,9619
3,8NHA23f7LvU,Jason Momoa Wows Hugh Grant With Some Dothraki...,24,2017-11-10T19:06:23.000Z,1496225,16116,236,605
4,fy-CuCzaPp8,Rooster Teeth Animated Adventures - Drunk Baby...,1,2017-11-13T14:00:03.000Z,308568,19541,70,495


In [43]:
mergedCA_df.to_sql(name="video_category_ca", con=engine, if_exists='append',index=False)
pd.read_sql_query('select * from video_category_ca', con=engine).head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments,category_title,country
0,0yIWz1XEeyc,Jake Paul Says Alissa Violet CHEATED with LOGA...,25,2017-11-13T07:37:51.000Z,1309699,103755,4613,12143,News & Politics,CA
1,TIvI07xLPnA,"The National for Sunday, November 12, 2017",25,2017-11-13T03:06:10.000Z,13433,74,57,0,News & Politics,CA
2,RbcGCMtlluQ,Final Nail In The Coffin: SNL Turns Against Th...,25,2017-11-13T05:46:39.000Z,116344,5068,106,1341,News & Politics,CA
3,TUmyygCMMGA,Why the rise of the robots won’t mean the end ...,25,2017-11-13T13:45:16.000Z,256426,12656,1363,2368,News & Politics,CA
4,2B8RO0GcLDw,Live with Dr.Shahid Masood | Asif Zardari | Na...,25,2017-11-12T16:23:14.000Z,187075,2443,185,324,News & Politics,CA


In [44]:
new_videoUS_df.to_sql(name = 'video_us', con = engine, if_exists = "append", index = False)
pd.read_sql_query('SELECT * FROM video_US', con = engine).head()


,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments
0,9wRQljFNDW8,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,17,2017-11-13T02:05:26.000Z,81377,655,25,177
1,Om_zGhJLZ5U,TL;DW - Every DCEU Movie Before Justice League,1,2017-11-12T18:00:03.000Z,288922,7515,792,2111
2,goP4Z5wyOlM,Iraq-Iran earthquake: Deadly tremor hits borde...,25,2017-11-12T21:16:40.000Z,34785,308,26,413
3,STI2fI7sKMo,"AFFAIRS, EX BOYFRIENDS, $18MILLION NET WORTH -...",22,2017-11-11T15:00:03.000Z,321053,4451,1772,895
4,ogYum4kWXgk,People are Awesome & The Pet Collective presen...,24,2017-11-13T13:00:06.000Z,69844,3417,33,160


In [45]:
mergedUS_df.to_sql(name="video_category_us", con=engine, if_exists='append',index=False)
pd.read_sql_query('select * from video_category_us', con=engine).head()

,video_id,title,category_id,publish_datetime,views,likes,dislikes,comments,category_title,country
0,9wRQljFNDW8,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,17,2017-11-13T02:05:26.000Z,81377,655,25,177,Sports,US
1,fIQ2Ty2OL34,James Harden Triple-Double 2017.11.09 vs Cavs ...,17,2017-11-10T04:00:52.000Z,18573,420,10,236,Sports,US
2,agKDPntMv-E,Stephen A. Smith slams the Philadelphia 76ers ...,17,2017-11-08T15:59:06.000Z,470191,4037,430,1553,Sports,US
3,Xl38SuxyrgM,"STEPH CURRY and OMRI CASSPI, postgame GSW (9-3...",17,2017-11-09T07:27:32.000Z,16305,147,13,47,Sports,US
4,0zk2SwmCpss,Joel Embiid Full Highlights 2017.11.13 at Clip...,17,2017-11-14T06:24:27.000Z,63780,831,12,301,Sports,US
